# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096850


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:51,  3.84s/it]

Rendering models:  10%|▉         | 3/31 [00:08<01:18,  2.81s/it]

Rendering models:  16%|█▌        | 5/31 [00:09<00:55,  2.12s/it]

Rendering models:  19%|█▉        | 6/31 [00:09<00:40,  1.62s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:21<00:01,  1.57it/s]

Rendering models:  94%|█████████▎| 29/31 [00:22<00:01,  1.79it/s]

Rendering models:  97%|█████████▋| 30/31 [00:22<00:00,  1.98it/s]

Rendering models: 100%|██████████| 31/31 [00:22<00:00,  2.11it/s]

DrJackie                              0.003751
CoreyGallegos23                       0.000849
Quarkgc                               0.000178
jnarayanbvg                           0.004788
Jonas_Cross                           0.000694
not-logged-in-6211468944fa611b8924    0.000189
not-logged-in-c6917d9b19f41e6c2995    0.000598
thiagoxxxx                            0.000350
not-logged-in-8998bdc9c49d01c69c85    0.038424
not-logged-in-5f30b2bb9b7f9d49da41    0.001899
not-logged-in-5f30b2bb9b7f9d49da41    0.003064
not-logged-in-dddbb97cfd84354416f5    0.000993
databanana                            0.008316
Chrisle206                            0.000669
ShirleyL                              0.000173
Lavadude                              0.077853
Paigery321                            0.001151
BrowardDaniel                         0.012211
clars915                              0.000778
Charly10                              0.001339
maxe83                                0.001320
ykuzmits     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())